In this example, we will use tensorflow.keras package to create a keras image classification application using model MobileNetV2, and transfer the application to Cluster Serving step by step.

### Original Keras application
We will first show an original Keras application, which download the data and preprocess it, then create the MobileNetV2 model to predict.

In [1]:
import tensorflow as tf
import os
import PIL

In [2]:
tf.__version__

'2.2.0'

In [3]:
# Obtain data from url:"https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip",
                                   fname="cats_and_dogs_filtered.zip", extract=True)

# Find the directory of validation set
base_dir, _ = os.path.splitext(zip_file)
test_dir = os.path.join(base_dir, 'validation')
# Set images size to 160x160x3
image_size = 160

# Rescale all images by 1./255 and apply image augmentation
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow images using generator to the test_generator
test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(image_size, image_size),
                batch_size=1,
                class_mode='binary')

Found 1000 images belonging to 2 classes.


In [4]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE=(160,160,3)
model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In keras, input could be ndarray, or generator. We could just use `model.predict(test_generator)`. But to simplify, here we just input the first record to model.

In [7]:
prediction=model.predict(test_generator.next()[0])
print(prediction)

[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    1.3335862e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    2.3917289e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 1.5355210e+00 ... 0.0000000e+00
    1.7352901e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 1.5430450e-03 ... 0.0000000e+00
    2.6985502e-01 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    7.0699120e-01 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 2.3603735e+00 ... 0.0000000e+00
    2.1094999e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 1.4562187e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.000000

Great! Now the Keras application is completed. 

### Export TensorFlow Saved Model
Next, we transfer the application to Cluster Serving. The first step is to save the model to SavedModel format.

In [9]:
# Save trained model to ./transfer_learning_mobilenetv2
model.save('transfer_learning_mobilenetv2')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: transfer_learning_mobilenetv2/assets


### Deploy Cluster Serving
After model prepared, we start to deploy it on Cluster Serving.

We config the model path in `config.yaml` to following (if you do not know what is `config.yaml`, check [Cluster Serving Configuration](https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#2-configuration))
```
## Analytics-zoo Cluster Serving

model:
  # model path must be provided
  path: /path/to/transfer_learning_mobilenetv2
```


After configuration, start Cluster Serving by `cluster-serving-start` as mentioned in [Cluster Serving Programming Guide](https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#3-launching-service)

Next we start Cluster Serving code

In [ ]:
from zoo.serving.client import InputQueue, OutputQueue
input_queue = InputQueue()

In Cluster Serving, only NdArray is supported as input. Thus, we first transform the generator to ndarray (If you do not know how to transform your input to NdArray, you may get help at [data transform guide](https://github.com/intel-analytics/analytics-zoo/tree/master/docs/docs/ClusterServingGuide/OtherFrameworkUsers#data))

In [11]:
arr = test_generator.next()[0]
arr

array([[[[0.04705883, 0.        , 0.07450981],
         [0.06666667, 0.05882353, 0.21568629],
         [0.57254905, 0.68235296, 0.8705883 ],
         ...,
         [0.5176471 , 0.38823533, 0.21960786],
         [0.4784314 , 0.3529412 , 0.19215688],
         [0.48627454, 0.37254903, 0.20784315]],

        [[0.12941177, 0.09803922, 0.25882354],
         [0.38431376, 0.37647063, 0.52156866],
         [0.8862746 , 0.9215687 , 0.9803922 ],
         ...,
         [0.4666667 , 0.36862746, 0.21176472],
         [0.45882356, 0.3647059 , 0.21568629],
         [0.45098042, 0.3647059 , 0.21960786]],

        [[0.4901961 , 0.52156866, 0.5294118 ],
         [0.54901963, 0.5529412 , 0.52156866],
         [0.5294118 , 0.49803925, 0.35686275],
         ...,
         [0.41960788, 0.34509805, 0.21960786],
         [0.43529415, 0.37254903, 0.24313727],
         [0.4156863 , 0.35686275, 0.23529413]],

        ...,

        [[0.34901962, 0.47450984, 0.57254905],
         [0.37647063, 0.5019608 , 0.6       ]

In [ ]:
# Use async api to put and get, you have pass a name arg and use the name to get
input_queue.enqueue('my-input', arr)
output_queue = OutputQueue()
prediction = output_queue.query('my-input')
# Use sync api to predict, this will block until the result is get or timeout
prediction = input_queue.predict(arr)

If everything works well, the result `prediction` would be the exactly the same NdArray object with the output of original Keras model.

This is the end of this tutorial. If you have any question, you could raise an issue at [Analytics Zoo Github](https://github.com/intel-analytics/analytics-zoo/issues).